**Description:**
This notebook is designed to update a collection in MongoDB. The script connects to a MongoDB database, fetches a specific collection, and updates records based on a provided data set.

In [ ]:
# Required Libraries
from pymongo import MongoClient, UpdateOne
from bson.objectid import ObjectId

# Connect to your MongoDB database
client = MongoClient('mongodb://your_mongodb_uri')
db = client.your_db_name  # replace 'your_db_name' with the name of your database
collection = db.attractions

**Description:**
We have a dataset of attractions with specific fields to be updated in the database. Below is the dataset:

In [ ]:
data = [
    {"id": "552be710912913b260eb7638", "boroughCode": "MN", "neighborhood": "Morningside Heights", "postalCode": "10027"},
    {"id": "552aacc6912913b260eb7620", "boroughCode": "MN", "neighborhood": "Morningside Heights", "postalCode": "10027"},
]

**Description:**
Now, we'll loop through the dataset and update the records in the MongoDB collection.

In [ ]:
totalRecords = len(data)
recordsUpdated = 0

operations = []

for item in data:
    operations.append(
        UpdateOne(
            {"_id": ObjectId(item['id'])},
            {
                "$set": {
                    "loc.boroughCode": item['boroughCode'],
                    "loc.neighborhood": item['neighborhood'],
                    "loc.postalCode": item['postalCode']
                }
            },
            upsert=True
        )
    )

result = collection.bulk_write(operations)
recordsUpdated = result.modified_count + len(result.upserted_ids)

recordsUpdated

**Description:**
Print out the results, showing the total number of records processed and the number of records updated.

In [ ]:
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")